# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas e linhas do DataFrame
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None) 

# Base de dados

In [4]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini.csv"

caminho_2 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(1).csv"
#
caminho_3 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"
#
caminho_4 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"
#
caminho_5 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(4).csv"
#
#caminho_6 = r"D:\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(5).csv"



# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)
df5 = pd.read_csv(caminho_5)
#df6 = pd.read_csv(caminho_6)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Inserindo _ nos titulos das colunas
df.columns = df.columns.str.replace(' ', '_')

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)

# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [5]:
df_saldos = df[['ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [6]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 40.221.277,73
O total de saldo a liquidar pela ESA até o momento foi: R$ 7.822.745,58
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 528.750,32
O total de saldo a pagar pela ESA até o momento foi: R$ 8.351.495,90


In [7]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_white')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [8]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [9]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [10]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [11]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [12]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [13]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [14]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

# Suporte para o Dashboard

In [15]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
461,2023NE000034,68.87,676,PASSAGENS PARA O PAIS,33903301
460,2023NE000033,3236.21,676,PASSAGENS PARA O PAIS,33903301
462,2023NE000329,330.22,617,PASSAGENS PARA O PAIS,33903301
2679,2023NE001680,3466.66,417,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
2676,2023NE001672,5543.33,417,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,33903917
2678,2023NE001678,4794.00,417,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
2680,2023NE001681,5902.08,417,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
372,2023NE001739,15330.52,384,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
32,2024NE000006,497.61,333,SERVICOS DE TELECOMUNICACOES,33903958
2539,2024NE000011,15101.43,322,PASSAGENS PARA O PAIS,33903301


In [16]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [17]:
# Qtde de empenhos realizados até o momento
df.shape[0]

4724

# Requisição em API para consulta a CNPJ

In [18]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/03126110000192"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '04/05/1999',
 'atividade_principal': [{'code': '49.30-2-04',
                          'text': 'Transporte rodoviário de mudanças'}],
 'atividades_secundarias': [{'code': '49.30-2-01',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'municipal.'},
                            {'code': '49.30-2-02',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'intermunicipal, interestadual e '
                                     'internacional'},
                            {'code': '52.11-7-02', 'text': 'Guarda-móveis'},
                            {'code': '68.10-2-02',
                             'text': 'Aluguel de imóveis próprios'}],
 'bairro': 'ZONA INDUSTRIAL (GUARA)',
 'billing': {'database': True, 'fre

In [19]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [20]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [21]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [22]:
dicionario['telefone']

'(41) 3316-7700'

In [23]:
dicionario['email']

'fiscal@positivo.com.br'

In [24]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [25]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['FAV'] != '160129']

df_cnpj['FAV'].unique()

array(['20584944000160', '39750831000116', '13081240000100', ...,
       '37902352000142', '02700680669', '07266248000148'], dtype=object)

In [26]:
df_cnpj[df_cnpj['FAV'] == '160129']

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [27]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente.head(2)

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
3,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,I3DA,CNT,LICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2024NE000004,2024,2024-01-10,160129000012024NE000004,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,335,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,0.0,0.00,0.00,103653.60
4,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,I3DA,CNT,LICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2024NE000071,2024,2024-02-14,160129000012024NE000071,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,300,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,339.0,28690.84,29029.84,183880.73


# Pesquisas no DataFrame

In [28]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos inscritos em RPNP

In [29]:
# Criando um Dataframe para filtrar os empenhos em RPNP
df_restos = df_teste[df_teste['ANO'].isin(['2022', '2023'])]
df_restos = df_restos[df_restos['A_LIQUIDAR'] !=  0]

### Restos a pagar ND 30 e 52

In [30]:
df_restos_material = df_restos[df_restos['ND'].isin(['339030', '449052' ])]
df_restos_material = df_restos_material.sort_values(by='A_LIQUIDAR', ascending=False)
df_restos_material.head()
df_restos_material.to_excel('restos_a_pagar_2024.xlsx')

In [31]:
cols = ['UG', 'NOME_ACAO', 'ANO', 'NOME_CREDOR',
'DATA', 'DIAS', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'PI', 
'NOME_PI', 'TIPO', 'TIPO_NE', 'NOME_UGR', 'A_LIQUIDAR', 
'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_filtro = df_teste.loc[:,cols]
df_filtro = df_filtro[df_filtro['A_LIQUIDAR'] != 0]
df_filtro.sort_values(by='DIAS', ascending=False).head(2)

,UG,NOME_ACAO,ANO,NOME_CREDOR,DATA,DIAS,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,TIPO,TIPO_NE,NOME_UGR,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
461,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,ORLEANS VIAGENS E TURISMO LTDA,2023-02-03,676,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000034,C4ENMILCAPE,CAPACITACAO DE PESSOAL,RPNP,GLOBAL,DECEX/GESTOR,68.87,119.88,188.75,9629.00
460,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,FACTO TURISMO LTDA,2023-02-03,676,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000033,C4ENMILCAPE,CAPACITACAO DE PESSOAL,RPNP,GLOBAL,DECEX/GESTOR,3236.21,0.00,3236.21,19261.89


### Filtros Diversos

In [32]:
df_liquidar = df_teste[df_teste['A_LIQUIDAR'] != 0].sort_values(by= 'DIAS', ascending= False).reset_index(drop=True)

In [33]:
cols = ['UG','NOME_ACAO', 'NOME_UGR', 'PI',
       'NOME_PI', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'ANO', 'DATA',
       'FAV', 'NOME_CREDOR', 'UF_FAV', 'TIPO', 'DIAS', 'UF_UG',
       'MOD_LIC', 'TIPO_NE', 'RP', 'NOME_RP', 'UO', 'NOME_UO', 'A_LIQUIDAR',
       'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_liquidar.loc[:, cols].head()

,UG,NOME_ACAO,NOME_UGR,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,MOD_LIC,TIPO_NE,RP,NOME_RP,UO,NOME_UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000034,2023,2023-02-03,21331404000138,ORLEANS VIAGENS E TURISMO LTDA,SP,RPNP,676,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,68.87,119.88,188.75,9629.00
1,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000033,2023,2023-02-03,14807420000199,FACTO TURISMO LTDA,SP,RPNP,676,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,3236.21,0.00,3236.21,19261.89
2,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000329,2023,2023-04-03,14807420000199,FACTO TURISMO LTDA,SP,RPNP,617,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,330.22,0.00,330.22,3828.26
3,167129,ADMINISTRACAO DA UNIDADE,DECEX - GESTOR,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2023NE001672,2023,2023-10-20,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,MG,RPNP,417,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52921,FUNDO DO EXERCITO,5543.33,0.00,5543.33,27716.65
4,167129,ADMINISTRACAO DA UNIDADE,DECEX - GESTOR,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,339040,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,33904012,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,2023NE001681,2023,2023-10-20,38680138000151,GESMAQ COMERCIO E SERVICOS LTDA,MG,RPNP,417,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52921,FUNDO DO EXERCITO,5902.08,5902.08,11804.16,23608.32


In [34]:
df_liquidar.to_excel('empenhos_2024_liquidar.xlsx')

In [35]:
df_empenhos_mais_60_dias = df_teste[(df_teste['DIAS'] >= 60)& (df_teste['A_LIQUIDAR'] != 0)].sort_values(by="A_LIQUIDAR", ascending=False)

df_empenhos_mais_60_dias.to_excel('empenhos_mais_60_dias.xlsx')

In [36]:
df_ensino = df_teste[df_teste['AÇÃO'] == '8965']

In [37]:
total_pago_ensino = df_ensino['PAGO'].sum().round(2)
total_a_liquidar_ensino = df_ensino['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar_ensino = df_ensino['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar_ensino = df_ensino['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago_ensino, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar_ensino, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar_ensino, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar_ensino, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 1.864.095,94
O total de saldo a liquidar pela ESA até o momento foi: R$ 1.528.558,37
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 87.640,22
O total de saldo a pagar pela ESA até o momento foi: R$ 1.616.198,59


In [38]:
df_ensino.sort_values(by='A_LIQUIDAR', ascending=False).head(5)

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
2548,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903309,TRANSPORTE DE SERVIDORES,2024NE000810,2024,2024-07-03,160129000012024NE000810,05824788000174,EMPRESA DE TRANSPORTES KOPERECK LTDA,RS,ANO,160,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,485091.90,565.97,485657.87,264344.65
2550,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903309,TRANSPORTE DE SERVIDORES,2024NE001947,2024,2024-11-13,160129000012024NE001947,05824788000174,EMPRESA DE TRANSPORTES KOPERECK LTDA,RS,ANO,27,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,199240.90,0.00,199240.90,0.00
2557,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903922,"EXPOSICOES, CONGRESSOS E CONFERENCIAS",2024NE001570,2024,2024-11-07,160129000012024NE001570,31741500000108,ELETROVIX COMERCIO IMPORTACAO E SERVICOS LTDA,ES,ANO,33,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,173210.00,0.00,173210.00,0.00
2560,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2024NE002564,2024,2024-12-06,160129000012024NE002564,15272846000158,PS SERVICOS DE LAVANDERIA LTDA,RN,ANO,4,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,164024.64,0.00,164024.64,0.00
2555,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2024NE002565,2024,2024-12-06,160129000012024NE002565,53202768000104,TOALL COMERCIO E SERVICOS LTDA.,SP,ANO,4,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,128001.30,0.00,128001.30,0.00


In [39]:
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df_ensino.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
#df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]

df_aux.to_excel('ensino_recursos_2024.xlsx')

In [40]:
df_transporte = df_teste[df_teste['ND'] == '339033']

total_pago_ensino = df_transporte['PAGO'].sum().round(2)
total_a_liquidar_ensino = df_transporte['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar_ensino = df_transporte['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar_ensino = df_transporte['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago_ensino, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar_ensino, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar_ensino, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar_ensino, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 1.602.457,93
O total de saldo a liquidar pela ESA até o momento foi: R$ 1.229.142,22
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 2.913,67
O total de saldo a pagar pela ESA até o momento foi: R$ 1.232.055,89


In [41]:
df_transporte.to_csv('nd33_2024.csv')

In [42]:
df_discricionarias = df_teste[(df_teste['AÇÃO'] == '2000') & (df_teste['A_LIQUIDAR'] != 0)]

In [43]:
df_teste[(df_teste['AÇÃO'] == '212O') & (df_teste['A_LIQUIDAR'] != 0)]

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
580,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000362,2024,2024-04-10,160129000012024NE000362,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,ANO,244,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,19850.00,0.0,19850.00,0.0
581,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000717,2024,2024-06-20,160129000012024NE000717,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,ANO,173,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,19850.00,0.0,19850.00,0.0
582,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000718,2024,2024-06-20,160129000012024NE000718,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,ANO,173,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,34684.00,0.0,34684.00,0.0
583,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000719,2024,2024-06-20,160129000012024NE000719,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,ANO,173,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,50049.00,0.0,50049.00,0.0
584,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000742,2024,2024-06-25,160129000012024NE000742,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,ANO,168,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,21850.00,0.0,21850.00,0.0
585,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000842,2024,2024-07-14,160129000012024NE000842,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,ANO,149,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,21850.00,0.0,21850.00,0.0
586,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000843,2024,2024-07-15,160129000012024NE000843,39644514000115,MULTI PRIME M